In [112]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd


In [157]:
df = pd.read_csv("data.csv")
df.head()

,Loan Account,Principal-M,Rate-of-interest,EMI,Earlier-due-date,New-due-date
0,FSAPLALONS000005000073,212739.43,25.73,9465,7,30
1,FSAPLALONS000005000072,37990.00,26.50,5917,7,30


In [169]:
part_payment = pd.read_csv("pre-payment.csv")
part_payment.head()

,Loan Account,Receipt Number,PART_PREPAY_AMOUNT,REPAY_EFFECTIVE_DATE
0,FSAPLALONS000005000115,6458210,729,11-04-2022
1,FSAPLALONS000005000330,6490591,208,30-04-2022
2,FSAPLALONS000005000407,6574372,866,23-06-2022
3,FSAPLALONS000005000487,6472702,598,20-04-2022
4,FSAPLALONS000005000073,6472722,1000,11-04-2022


In [179]:
part_payment.head()

,Loan Account,Receipt Number,PART_PREPAY_AMOUNT,REPAY_EFFECTIVE_DATE,REPAY_EFFECTIVE_DATE_D
0,FSAPLALONS000005000115,6458210,729,2022-11-04,2022-11-04
1,FSAPLALONS000005000330,6490591,208,2022-04-30,2022-04-30
2,FSAPLALONS000005000407,6574372,866,2022-06-23,2022-06-23
3,FSAPLALONS000005000487,6472702,598,2022-04-20,2022-04-20
4,FSAPLALONS000005000073,6472722,1000,2022-11-04,2022-11-04


In [171]:
pre_payment_list=part_payment.loc[part_payment['Loan Account']=="FSAPLALONS000005000073",'PART_PREPAY_AMOUNT'].values.tolist()
print(pre_payment_list)

[1000]


In [172]:
repay_date_list =part_payment.loc[part_payment['Loan Account']=="FSAPLALONS000005000073",'REPAY_EFFECTIVE_DATE'].values.tolist()
print(repay_date_list)


['11-04-2022']


In [182]:
repay_date = repay_date_list[0].split("-")[0]

'11'

In [175]:
part_payment['REPAY_EFFECTIVE_DATE'] = pd.to_datetime(part_payment['REPAY_EFFECTIVE_DATE'])

In [181]:
import datetime
d = part_payment['REPAY_EFFECTIVE_DATE'].dt.date.tolist()
d
# part_payment['REPAY_EFFECTIVE_DATE_D'] = [datetime.datetime.date(d) for d in part_payment['REPAY_EFFECTIVE_DATE']] 


[datetime.date(2022, 11, 4),
 datetime.date(2022, 4, 30),
 datetime.date(2022, 6, 23),
 datetime.date(2022, 4, 20),
 datetime.date(2022, 11, 4)]

In [167]:
type(result)

numpy.ndarray

In [40]:
principal_list = df['Principal-M'].to_list()
roi_list = df['Rate-of-interest'].to_list()
emi_list = df['EMI'].to_list()
edd_list = df['Earlier-due-date'].to_list()

In [41]:
d = edd_list[0]
sd = pd.to_datetime(f"202204{d}", format='%Y%m%d', errors='ignore')


In [42]:
d = 6
ed = pd.to_datetime(f"202204{d}", format='%Y%m%d', errors='ignore')


In [43]:
sd

Timestamp('2022-04-07 00:00:00')

In [44]:
principal_list[0]

212739.43

In [45]:
ed

Timestamp('2022-04-06 00:00:00')

In [46]:
roi_list[0]

25.73

In [47]:
def days360(start_date, end_date, method_eu=False):
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)

In [79]:


# first_interest = principal_list[0] * (roi_list[0]/1200)
# first_interest
# principal_apposition = emi_list[0]-first_interest
# closing_principal = principal_list[0] - principal_apposition
# print(closing_principal)

In [ ]:
#prepayment 


In [113]:
end_of_month_list = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [119]:
end_of_month["6"]

30

In [129]:
end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']

In [145]:
end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
interest_value_list = []

def first_closing_prinicipal(principal, roi, emi):
    first_interest = round(principal * (roi/1200),2)
    interest_value_list.append(first_interest)
    print("first interest",round(first_interest, 2))
    principal_apposition = emi - first_interest
    print("first prinicipal appo",round(principal_apposition, 2))
    closing_principal = principal - principal_apposition
    print("first closing prinicipal",round(closing_principal, 2))
    return closing_principal

def next_values(principal, roi, emi, date, sm, em):

    if sm <= 6:
        interest_value = round(principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')),2)
        # print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
        # print("sm, em", sm, em)
        # print("date",date)
        print("interest value", round(interest_value,2))
        interest_value_list.append(interest_value)
        principal_apposition = emi - interest_value
        print("principal apposition", round(principal_apposition,2))
        principal = principal - principal_apposition
        print("closing principal", round(principal,2))
        if sm==6:
            print("closing principal inside june before interest", round(principal,2))
            interest_value = round(principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-6-{date}'), pd.to_datetime(f'2022-6-30')),2)
            interest_value_list.append(interest_value)
            print("inside june interest", round(interest_value,2))
            principal = principal + interest_value
            print("june closing principal", round(principal,2))
    if sm>6:
        print("closing principal after june before interest", round(principal,2))
        interest_value = round(principal * (roi/ 36000) * days360(pd.to_datetime(f"{end_of_month_dates[sm-1]}"), pd.to_datetime(f"{end_of_month_dates[sm]}")),2)

        # print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
        # print("sm, em", sm, em)
        # print("date",date)
        print("interest value", round(interest_value,2))
        interest_value_list.append(interest_value)

        principal_apposition = emi - interest_value
        print("principal apposition", round(principal_apposition,2))
        principal = principal - principal_apposition
        print("closing principal", round(principal,2))
        
        # print("june closing principal", round(closing_principal,2))
    # if sm==8:
    #     # print("closing principal inside june before interest", round(closing_principal,2))
    #     interest_value = closing_principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-6-{date}'), pd.to_datetime(f'2022-6-30'))
    #     # print("inside june", round(interest_value,2))
    #     closing_principal = closing_principal + interest_value
    #     # print("june closing principal", round(closing_principal,2))
    # if sm==9:
    #     # print("closing principal inside june before interest", round(closing_principal,2))
    #     interest_value = closing_principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-6-{date}'), pd.to_datetime(f'2022-6-30'))
    #     # print("inside june", round(interest_value,2))
    #     closing_principal = closing_principal + interest_value
    #     # print("june closing principal", round(closing_principal,2))

    return interest_value,principal

In [146]:
closing_principal = first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(closing_principal, roi_list[0], emi_list[0], edd_list[0], i, i+1)
    closing_principal = closing_principal_2
    # else:
    #     break
print(round(interest_value,2))

first interest 4561.49
first prinicipal appo 4903.51
first closing prinicipal 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
interest value 4348.95
principal apposition 5116.05
closing principal 197711.22
closing principal inside june before interest 197711.22
inside june interest 3250.1
june closing principal 200961.32
closing principal after june before interest 200961.32
interest value 4308.95
principal apposition 5156.05
closing principal 195805.27
closing principal after june before interest 195805.27
interest value 4198.39
principal apposition 5266.61
closing principal 190538.66
closing principal after june before interest 190538.66
interest value 4085.47
principal apposition 5379.53
closing principal 185159.13
4085.47


In [147]:
print(interest_value_list)

[4561.49, 4456.35, 4348.95, 3250.1, 4308.95, 4198.39, 4085.47]


In [152]:
df['interest_april1'] = ""

In [153]:
df['interest_april1'].loc[0] = interest_value_list[0]

/home/laptop-pr-14/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [154]:
df['april_interest'] = ""
df['may_interest'] = ""
df['june_interest'] = ""
df['june_interest'] = ""
df['july_interest'] = ""
df['august_interest'] = ""
df['september_interest'] = ""

df['april_interest1'] = ""
df['may_interest1'] = ""
df['june_interest1'] = ""
df['july_interest1'] = ""
df['august_interest1'] = ""
df['september_interest1'] = ""

,Loan Account,Principal-M,Rate-of-interest,EMI,Earlier-due-date,New-due-date,Date,interest_april,interest_april1
0,FDSG898123,212739.43,25.73,9465,7,30,2022-07-31,4561.49,4561.49
1,FDSG898123,37990.00,26.50,5917,7,30,2022-07-31,4561.49,


In [ ]:

df['interest_april1'].loc[0] = interest_value_list[0]

In [106]:

df['Date'] = pd.to_datetime(f'2022-6-30') + MonthEnd(1)
df['Date'] 

0   2022-07-31
1   2022-07-31
Name: Date, dtype: datetime64[ns]

In [111]:
pd.to_datetime(f'2022-6-23') + MonthEnd(0)

Timestamp('2022-06-30 00:00:00')

In [50]:
first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])

4561.49


207835.91794491667

In [51]:
def pre_payment(closing_principal, payment):
    updated_closing_principal =  closing_principal - payment
    return updated_closing_principal


first interest 4561.49
first prinicipal appo 4903.51
first closing prinicipal 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
closing principal inside june before interest 202827.27
inside june interest 3334.2
june closing principal 206161.46
closing principal inside june before interest 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
closing principal inside june before interest 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
closing principal inside june before interest 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
4456.35


In [53]:
principal_list[0]
interest_value = 207835.478725 * (roi_list[0] / 36000) * days360(sd,ed)
print(interest_value)

-148.54463521095138


In [30]:
interest_value = 207835.478725 * (25.73/ 36000) * days360(pd.to_datetime('2022-04-20'), pd.to_datetime('2022-05-20'))
print(interest_value)

4456.339056328541


In [31]:
# decimal - done 
# june  interest value - done
# prepayment logic 
# sample data verification

In [32]:
days360(sd,ed)

-1

In [33]:
days360(pd.to_datetime('2022-04-20'), pd.to_datetime('2022-05-20'))

30

In [34]:
days360(pd.to_datetime('2012-11-27'), pd.to_datetime('2017-11-27'))

1800